In [12]:
%load_ext autoreload
%autoreload 2

from iob_ia.utils import segment, io_utils
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
path_tif = 'G:/20241120_IOB_Magdalena/3D_project/20240911_Hoechst 405_proA7 gfp 488_Car568_Cralbp647_BF_10x rapiclear Org2_01____TESTs/small_test_image.tif'
path_tif2 = 'G:/20241120_IOB_Magdalena/3D_project/20240911_Hoechst 405_proA7 gfp 488_Car568_Cralbp647_BF_10x rapiclear Org2_01____TESTs/small_crop_multiCh.tif'
path_vsi = 'G:/20241120_IOB_Magdalena/3D_project/20240911_Hoechst 405_proA7 gfp 488_Car568_Cralbp647_BF_10x rapiclear Org2_01.vsi'

In [3]:
# Specify the channels (0-based indexing)
dapi_ch = 0
gfp_ch = 1
# channels of interest
car = 2
cralbp = 3
# channel of no interest
bf = 4

In [4]:
img, voxel_size = io_utils.read_tif(path_tif2)

In [5]:
print(img.shape)
print(voxel_size)

(5, 313, 208, 248)
(1.99, 0.6500002275000796, 0.6500002275000796)


In [6]:
img_gfp = img[gfp_ch]
print(img_gfp.shape)

(313, 208, 248)


img2, voxel_size2 = io_utils.read_vsi(path_vsi)

print(img2.shape)
print(voxel_size2)

## Segment using cellpose

In [10]:
if os.path.exists(io_utils.gen_out_path(path_tif2, 'cp_seg')):
    m = io_utils.read_labels(io_utils.gen_out_path(path_tif2, 'cp_seg'))
else:
    m = segment.segment_3d_cellpose(img_gfp, anisotropy=voxel_size[0]/voxel_size[1])
print(m.shape)

(313, 208, 248)


In [ ]:
# save labels
save_path = io_utils.gen_out_path(path_tif2, 'cp_seg')
io_utils.save_labels(m, save_path)

save_path = io_utils.gen_out_path(path_tif2, 'gfpCh')
io_utils.save_labels(img_gfp, save_path)

In [ ]:
m.shape

## Filter lable image on features
shape features: 'area, 'euler_number' and 'extent'
intensity features: "intensity_max", "intensity_mean", "intensity_min"

In [61]:
lbls = segment.filter_shape(m, 'area', min_val=400)
#print(lbls)

In [62]:
lbls = segment.filter_intensity(m, img_gfp, 'intensity_mean', min_val=4000, labels_to_remove=lbls)
#print(lbls)

In [63]:
filtered_mask = segment.remove_label_objects(m, lbls)

100%|███████████████████████████████████████████████████████████████████████████████| 276/276 [00:01<00:00, 200.95it/s]


In [60]:
save_path = io_utils.gen_out_path(path_tif2, 'filtered')
io_utils.save_labels(filtered_mask, save_path)

Saved labels to: G:/20241120_IOB_Magdalena/3D_project/20240911_Hoechst 405_proA7 gfp 488_Car568_Cralbp647_BF_10x rapiclear Org2_01____TESTs\small_crop_multiCh_filtered.tif
